# Dependencies

In [1]:
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from config import api_key
import time
from pprint import pprint

In [2]:
import sqlalchemy
from sqlalchemy import create_engine, inspect, Column, Integer, Float, String, Sequence, Boolean
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base

In [3]:
musixmatch = Musixmatch(api_key)

# Setting up SQLlite

In [5]:
#Using declarative base
Base = declarative_base()
engine = create_engine('sqlite:///lyrics3.db', echo = False)

In [6]:
#defining the Tracks table
class Tracks(Base):
    __tablename__ = "tracks"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    album_id = Column(Integer)
    artist_id = Column(Integer)
    artist_name = Column(String)
    explicit = Column(Integer)
    has_lyrics = Column(Integer)
    lyrics_id = Column(Integer)
    track_id = Column(Integer, unique = True)
    track_name = Column(String)

In [7]:
#defining the Lyrics table
class Lyrics(Base):
    __tablename__ = "lyrics"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    explicit = Column(Integer)
    lyrics_body = Column(String)
    lyrics_id = Column(Integer, unique = True)
    lyrics_language = Column(String)
    lyrics_language_description = Column(String)

In [8]:
#defining the Artists table
class Artists(Base):
    __tablename__ = "artists"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer, unique = True)
    artist_name = Column(String)
    artist_q = Column(String)

In [9]:
#defining the Albums table
class Albums(Base):
    __tablename__ = "albums"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer)
    artist_name = Column(String)
    album_id = Column(Integer, unique = True)
    album_name = Column(String)

In [25]:
#defining the genres table
class Album_Genres(Base):
    __tablename__ = "album_genres"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer)
    artist_name = Column(String)
    album_id = Column(Integer)
    genre = Column(String)
    genre_id = Column(Integer)

In [26]:
#Creating the table metadata
Base.metadata.create_all(engine)
session = Session(engine)

In [27]:
#Binding engine to the metadata
meta = sqlalchemy.MetaData()
meta.reflect(bind = engine)

# MusixMatch API - Finding artists via text search from manually created list

In [11]:
artists_path = 'artist_names.xlsx'
artists_df = pd.read_excel(artists_path, sheetname = "Sheet1", header = None)
artists = list(artists_df[0].values)

In [12]:
#Connecting to tracks table
table = sqlalchemy.Table('artists', meta, autoload = True)

In [ ]:
for artist in artists:
    print(f"getting artist names {artist}")
    result = musixmatch.artist_search(q_artist = artist, page = 1, page_size = 100, f_artist_id = '', f_artist_mbid = '')
    print('extracting results')
    artist_list = result['message']['body']['artist_list']
    artists = [artist['artist'] for artist in artist_list]
    print("creating package")
    for item in artists:
        package = {
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name'],
            'artist_q': artist
        }
        try:
            print("writing package to database")
            session.execute(table.insert(), package)
        except:
            next
    print("committing to database")
    session.commit()

# MusixMatch API - Finding Albums from Artists

In [26]:
artists = pd.read_sql('artists', engine)
artists.dropna(inplace = True)
artist_ids = list(artists['artist_id'].values)

In [27]:
table = sqlalchemy.Table('albums', meta, autoload = True)

In [ ]:
for artist_id in artist_ids:
    print(f"getting albums for artist {artist_id}")
    result = musixmatch.artist_albums_get(artist_id = artist_id, g_album_name = '', page = 1, page_size = 100, s_release_date = '')
    print("extracting the album_list")
    album_list = result['message']['body']['album_list']
    print("extracting individual albums")
    albums = [album['album'] for album in album_list]
    print("iterating through the list of albums")
    for item in albums:
        print("creating the package")
        package = {
            'album_id': item['album_id'],
            'album_name': item['album_name'],
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name']
        }
        try:
            print("writing to the database")
            session.execute(table.insert(), package)
        except:
            next
    print("comitting to the database")
    session.commit()

# MusixMatch API - Scraping Tracks From Charts

In [19]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [10]:
country_codes = ['AU', 'CA', 'IE', 'IM', 'NZ', 'GB', 'US']

In [ ]:
country_count = 1
for country in country_codes:
    
    print(f"Attempting country {country_count} for {country}")
    
    page_count = 1

    while page_count < 5:
        print(f"calling api attempt {page_count} for {country}")
        result = musixmatch.chart_tracks_get(page_count, 100, f_has_lyrics = True)
        tracks = result['message']['body']['track_list']
        track_list = [track['track'] for track in tracks]
        print(f"track list is {len(track_list)}")
        for track in track_list:
            package = {
                'album_id': track['album_id'],
                'artist_id': track['artist_id'],
                'artist_name': track['artist_name'],
                'explicit': track['explicit'],
                'has_lyrics': track['has_lyrics'],
                'lyrics_id': track['lyrics_id'],
                'track_id': track['track_id'],
                'track_name': track['track_name']
            }
            print(package['track_name'])
            print("writing to database")
            try:
                session.execute(table.insert(), package)
            except:
                next
        page_count += 1
        time.sleep(1)
        session.commit()
    
    country_count += 1

# MusixMatch API - finding tracks from albums

## Starting point is the tracks data from the charts

In [20]:
tracks = pd.read_sql_table('tracks', engine, index_col = "id", coerce_float = False)

In [21]:
album_ids = tracks['album_id'].values

In [24]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## Starting point is the album's data

In [15]:
albums = pd.read_sql_table('albums', engine, index_col = "id", coerce_float = False)
albums = albums.iloc[647:]
album_ids = list(albums['album_id'].values)

In [16]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## MusixMatch API - Scraping Lyrics from the Albums Data

In [28]:
#Connecting to lyrics table
table = sqlalchemy.Table('lyrics', meta, autoload = True)

In [29]:
tracks = pd.read_sql('tracks', engine, index_col = "id")
track_ids = tracks['track_id'].values

In [ ]:
count = 1
for track in track_ids:
    print(f"calling API attempt {count} for track_id# {track}")
    print(f"track data type is {type(track)}")
    lyrics = musixmatch.track_lyrics_get(track)
    print("pulling out lyrics package")
    lyrics_package = lyrics['message']['body']['lyrics']
    print("pulling out package to insert into SQL database")
    package = {
        "explicit": lyrics_package['explicit'],
        "lyrics_body": lyrics_package["lyrics_body"][:-69],
        "lyrics_id": lyrics_package['lyrics_id'],
        "lyrics_language": lyrics_package['lyrics_language'],
        "lyrics_language_description": lyrics_package['lyrics_language_description'],
        "updated_time": lyrics_package['updated_time']
    }
    
    print(package['lyrics_body'][:10])
    print("writing to database")
    try:
        session.execute(table.insert(), package)
    except:
        next
    print("committing files to database")
    session.commit()
    count += 1 

# Acquiring Genre from Album

In [ ]:
#Connecting to lyrics table
table = sqlalchemy.Table('genre', meta, autoload = True)

In [ ]:
albums = pd.read_sql_table('albums', engine, index_col = "id", coerce_float = False)
album_ids = list(albums['album_id'].values)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_get(album_id = album_id)
    print(f"getting information for {album_id}")
    album = result['message']['body']['album']
    print("creating genre list")
    genres = result['message']['body']['album']['primary_genres']['music_genre_list']
    genre_list = [genre['music_genre'] for genre in genres]
    print(f"Iterating through album genres")
    for genre in track_list:
        package = {
                'album_id': int(album['album_id']),
                'artist_id': int(album['artist_id']),
                'artist_name': album['artist_name'],
                'genre': genre['music_genre_name'],
                'genre_id': genre['music_genre_id']
                
            }
        print(package['album_id'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1